In [16]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.vectorstores import faiss as langchain_faiss
import faiss
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

In [2]:
def pdf_load(data_dir):
    loader = DirectoryLoader(data_dir, glob="*.pdf", loader_cls=PyPDFLoader)
    
    doc = loader.load()
    return doc
extdata = pdf_load("data_dir")

In [3]:
def split(extdata):
    text_split = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap = 80)
    tokens = text_split.split_documents(extdata)
    return tokens
tokens = split(extdata)
print("length of chunks:",len(tokens))

length of chunks: 75


In [4]:
def hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device': 'cpu'})
    return embeddings
embeddings = hf_embeddings()

c:\Users\LENOVO\OneDrive\Desktop\gitpost\medical_chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
ans_embed = embeddings.embed_query("pushkar sharma")
print(f"Embedding of 'pushkar sharma' is: {ans_embed}")
print(f"Length of the embedded word is: {len(ans_embed)}")


Embedding of 'pushkar sharma' is: [-0.006053785793483257, -0.022976543754339218, -0.07094717770814896, -0.018650084733963013, -0.027309833094477654, 0.0848015546798706, 0.09731242060661316, 0.037898700684309006, 0.05176715552806854, 0.037358447909355164, 0.01607447676360607, -0.15937912464141846, -0.0050665452145040035, 0.03932378068566322, 0.07719752192497253, 0.04647071287035942, -0.0017382897203788161, 0.004604054614901543, 0.041276853531599045, -0.0689355880022049, -0.06302502006292343, 0.04452357441186905, -0.03566081449389458, -0.04886932671070099, 0.006537001579999924, -0.026425981894135475, 0.020379597321152687, -0.001554164569824934, -0.05121106281876564, -0.0588819645345211, 0.08923058211803436, -0.05794423073530197, 0.022681085392832756, 0.05013495311141014, -0.05096560716629028, 0.032543666660785675, -0.07006102800369263, 0.10825872421264648, 0.04622697830200195, -0.05700882896780968, -0.013814974576234818, -0.10016147047281265, 0.03578101843595505, -0.014829195104539394, 0

In [7]:
DB_FAISS_PATH = 'vectorstore/db_faiss'

In [8]:
db = FAISS.from_documents(tokens, embeddings)
db.save_local(DB_FAISS_PATH)

In [9]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [10]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [11]:
llm=CTransformers(model="llm/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':512,'temperature':0.8})

In [12]:
from langchain.chains import RetrievalQA

In [13]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=db.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [14]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])


KeyboardInterrupt



In [26]:
query_text = "What is the focus of the present patent application?"
response = qa(query_text)
print(response)

combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nProvide a response based on the given information. \nIf you don't know the answer, simply state that you're unsure; do not attempt to fabricate an answer.\n\nContext: {context}\nQuestion: {question}\n\nUse the context and question provided below.\nHelpful answer:\n"), llm=CTransformers(client=<ctransformers.llm.LLM object at 0x000002AADFC3F8D0>, model='llm/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})), document_variable_name='context') return_source_documents=True retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002AAC9AF1790>, search_kwargs={'k': 2})
